In [1]:
# This file imports country data from the International Monetary Fund and International Country Risk Guide to 
# create predictor variables. It also imports the country bond rating from Fitch as the target prediction. 

In [2]:
import pandas as pd
import numpy as np

WEO = pd.read_excel("E:\Downloads\IMF\WEOApr2018all.xlsx")

In [3]:
PoliticalFile = pd.ExcelFile("E:\Downloads\IMF\Political2014.xlsx")
FinanicalFile = pd.ExcelFile("E:\Downloads\IMF\Finanical2014.xlsx")
EconomicFile = pd.ExcelFile("E:\Downloads\IMF\Economic2014.xlsx")
PNames = PoliticalFile.sheet_names
FNames = FinanicalFile.sheet_names
ENames = EconomicFile.sheet_names

In [4]:
Fitch = pd.read_excel("E:\Downloads\IMF\FitchRatings.xlsx")

In [5]:
Df = pd.DataFrame()

for i in range(0,len(PNames)):
    df = pd.read_excel(PoliticalFile, sheet_name=PNames[i])
    df['Indicator']=PNames[i]
    frames = [Df, df]
    Df = pd.concat(frames)

for i in range(0,len(FNames)):
    df = pd.read_excel(FinanicalFile, sheet_name=FNames[i])
    df['Indicator']=FNames[i]
    frames = [Df, df]
    Df = pd.concat(frames)

for i in range(0,len(ENames)):
    df = pd.read_excel(EconomicFile, sheet_name=ENames[i])
    df['Indicator']=ENames[i]
    frames = [Df, df]
    Df = pd.concat(frames)

frames = [Df,WEO]
Df = pd.concat(frames)


In [6]:
RatingScale = np.array(['AAA','AA+','AA','AA-','A+','A','A-','BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-','CCC+','CCC','CCC-','CC','C','RD','DDD','DD','D'])
RatingScale = np.flip(RatingScale,axis = 0)
FitchCountry = Fitch['Country']
FitchYear = Fitch.loc[:,'Date'].apply(lambda x:x.year)
col=['Country', 'Year', 'RatingS']
FitchAArr = np.array([])

Country1 = FitchCountry[0]
Year1 = FitchYear[0]
Rating1 = Fitch.loc[(FitchCountry==Country1)&(FitchYear==Year1),'Foreign currency long-term']    
RatingT = np.zeros(len(Rating1))
MeanRating = np.nan
for j in range(0,len(Rating1)):
    Rating2 = Rating1.iloc[j]
    RatingT[j] = np.where(RatingScale==Rating1.iloc[j])[0] 
MeanRating=np.round(np.mean(RatingT)/5)
newrow = [Country1,Year1,MeanRating]
FitchAArr = np.array(newrow)
FitchAArr = np.vstack([FitchAArr, newrow])

for i in range(1,len(Fitch)):
    Country1 = FitchCountry[i]
    Year1 = FitchYear[i]
    Year2 = FitchAArr[FitchAArr[:,0] ==Country1,1]
    if not(Year1 in Year2.astype(np.int)):
        Rating1 = Fitch.loc[(FitchCountry==Country1)&(FitchYear==Year1),'Foreign currency long-term']    
        RatingT = np.zeros(len(Rating1))
        MeanRating = np.nan
        for j in range(0,len(Rating1)):
            Rating2 = Rating1.iloc[j]
            if Rating2 in RatingScale:
                RatingT[j] = np.where(RatingScale==Rating1.iloc[j])[0] 
            else:
                break
        MeanRating=np.round(np.mean(RatingT)/5)
        newrow = [Country1,Year1,MeanRating]
        FitchAArr = np.vstack([FitchAArr, newrow])
FitchA = pd.DataFrame(FitchAArr,columns=col)


In [7]:
X = np.zeros((len(FitchA), 21))
Y = np.zeros(len(FitchA))
AllNan = np.full(len(FitchA), True)

for i in range(0,len(FitchA)):
    Country1 = FitchA.iloc[i,0]
    Year1 = FitchA.iloc[i,1]
    Rating1 = FitchA.iloc[i,2]

    DfCY = Df[Df['Country'] == Country1]

    NGDPD = DfCY[DfCY['WEO Subject Code'] == 'NGDPD'][np.int(Year1)]
    if NGDPD.size>0:
        NGDPD = NGDPD.iat[0]
    else:
        NGDPD = np.nan
    NID_NGDP = DfCY[DfCY['WEO Subject Code'] == 'NID_NGDP'][np.int(Year1)]
    if NID_NGDP.size>0:
        NID_NGDP = NID_NGDP.iat[0]
    else:
        NID_NGDP = np.nan
    NGSD_NGDP = DfCY[DfCY['WEO Subject Code'] == 'NGSD_NGDP'][np.int(Year1)]
    if NGSD_NGDP.size>0:
        NGSD_NGDP = NGSD_NGDP.iat[0]
    else:
        NGSD_NGDP = np.nan
    GGXWDG_NGDP = DfCY[DfCY['WEO Subject Code'] == 'GGXWDG_NGDP'][np.int(Year1)]
    if GGXWDG_NGDP.size>0:
        GGXWDG_NGDP = GGXWDG_NGDP.iat[0]
    else:
        GGXWDG_NGDP = np.nan
    BCA_NGDPD = DfCY[DfCY['WEO Subject Code'] == 'BCA_NGDPD'][np.int(Year1)]
    if BCA_NGDPD.size>0:
        BCA_NGDPD = BCA_NGDPD.iat[0]
    else:
        BCA_NGDPD = np.nan
    LUR = DfCY[DfCY['WEO Subject Code'] == 'LUR'][np.int(Year1)]
    if LUR.size>0:
        LUR = LUR.iat[0]
    else:
        LUR = np.nan
    GovStab = DfCY[DfCY['Indicator'] == 'Government Stability'][np.int(Year1)]
    if GovStab.size>0:
        GovStab = GovStab.iat[0]
    else:
        GovStab = np.nan
    SocCond = DfCY[DfCY['Indicator'] == 'Socioeconomic Conditions'][np.int(Year1)]
    if SocCond.size>0:
        SocCond = SocCond.iat[0]
    else:
        SocCond = np.nan
    IntConf = DfCY[DfCY['Indicator'] == 'Internal Conflict'][np.int(Year1)]
    if IntConf.size>0:
        IntConf = IntConf.iat[0]
    else:
        IntConf = np.nan
    ExtConf = DfCY[DfCY['Indicator'] == 'External Conflict'][np.int(Year1)]
    if ExtConf.size>0:
        ExtConf = ExtConf.iat[0]
    else:
        ExtConf = np.nan
    BurQual = DfCY[DfCY['Indicator'] == 'Bureaucracy Quality'][np.int(Year1)]
    if BurQual.size>0:
        BurQual = BurQual.iat[0]
    else:
        BurQual = np.nan
    ForDebt = DfCY[DfCY['Indicator'] == 'ForDebt'][np.int(Year1)]
    if ForDebt.size>0:
        ForDebt = ForDebt.iat[0]
    else:
        ForDebt = np.nan
    XRStab = DfCY[DfCY['Indicator'] == 'XRStab'][np.int(Year1)]
    if XRStab.size>0:
        XRStab = XRStab.iat[0]
    else:
        XRStab = np.nan
    DebtServ = DfCY[DfCY['Indicator'] == 'DebtServ'][np.int(Year1)]
    if DebtServ.size>0:
        DebtServ = DebtServ.iat[0]
    else:
        DebtServ = np.nan
    CAXGS = DfCY[DfCY['Indicator'] == 'CAXGS'][np.int(Year1)]
    if CAXGS.size>0:
        CAXGS = CAXGS.iat[0]
    else:
        CAXGS = np.nan
    IntLiq = DfCY[DfCY['Indicator'] == 'IntLiq'][np.int(Year1)]
    if IntLiq.size>0:
        IntLiq = IntLiq.iat[0]
    else:
        IntLiq = np.nan
    Inflation = DfCY[DfCY['Indicator'] == 'Inflation'][np.int(Year1)]
    if Inflation.size>0:
        Inflation = Inflation.iat[0]
    else:
        Inflation = np.nan
    GDPHead = DfCY[DfCY['Indicator'] == 'GDPHead'][np.int(Year1)]
    if GDPHead.size>0:
        GDPHead = GDPHead.iat[0]
    else:
        GDPHead = np.nan
    GDPGrowth = DfCY[DfCY['Indicator'] == 'GDPGrowth'][np.int(Year1)]
    if GDPGrowth.size>0:
        GDPGrowth = GDPGrowth.iat[0]
    else:
        GDPGrowth = np.nan
    BudBal = DfCY[DfCY['Indicator'] == 'BudBal'][np.int(Year1)]
    if BudBal.size>0:
        BudBal = BudBal.iat[0]
    else:
        BudBal = np.nan
    CACC = DfCY[DfCY['Indicator'] == 'BudBal'][np.int(Year1)]
    if CACC.size>0:
        CACC = CACC.iat[0]
    else:
        CACC = np.nan

 
    X[i,:] = np.array([NGDPD,NID_NGDP,NGSD_NGDP,GGXWDG_NGDP,BCA_NGDPD,LUR,GovStab,SocCond,IntConf,ExtConf,BurQual,ForDebt,XRStab,DebtServ,CAXGS,IntLiq,Inflation,GDPHead,GDPGrowth,BudBal,CACC])
    Y[i] = Rating1
    if all(np.isnan(X[i,:])):
        AllNan[i] = False

X = X[AllNan,:]
Y = Y[AllNan]    


In [8]:
import pickle

print(np.shape(X))
f1 = open('X.pckl', 'wb')
pickle.dump(X, f2)
f1.close()

print(np.shape(Y))
f2 = open('Y.pckl', 'wb')
pickle.dump(Y, f3)
f2.close()

(719, 21)
(719,)
